In [1]:
import math
import numpy as np
import pandas as pd
import pygeohash as gh
import os

In [2]:
pwd

'/Users/joachimclodic/code/G-Dolle/DIVVY_BIKE/notebooks'

In [3]:
cd ..

/Users/joachimclodic/code/G-Dolle/DIVVY_BIKE


In [4]:
from ml_logic.main import preprocess,preprocess_test

In [5]:
# X_processed_df, y_processed_df, preprocessor=preprocess('nb_departures')
# X_processed_df.head()

In [6]:
from ml_logic.data_import import get_weather_data, get_divvy_data
from ml_logic.cleaning import compute_geohash_stations,weather_cleaning, cleaning_divvy_gen,cleaning_divvy_gen_agg, merge_divvy_weather, features_target
from ml_logic.preprocessor import transform_time_features, preprocess_features, target_process


In [26]:
X_processed_df, y_processed_df, preprocessor=preprocess('nb_departures')
X_processed_df.tail()

Raw data imported


/Users/joachimclodic/code/G-Dolle/DIVVY_BIKE/ml_logic/cleaning.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_reduced.rename(columns={"name":"station_name"}, inplace=True)


Data cleaned and merged
features and target dataframes created
features preprocessed
nb_departures picked as target
Preprocessing of Training set is done


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
5869,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.283048,-1.986207,1.680405,-0.808216,0.859744,0.0,0.0,0.0,0.0,1.0
5870,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-2.146075,-0.211851,2.156065,-0.404995,0.859744,0.0,0.0,0.0,0.0,1.0
5871,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-2.587734,1.710367,-0.850289,-0.072338,0.859744,0.0,0.0,0.0,0.0,1.0
5872,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-3.029393,1.858230,-0.360775,-0.122741,0.859744,0.0,0.0,0.0,0.0,1.0
5873,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-3.029393,1.562504,-0.647095,0.976035,0.859744,0.0,0.0,0.0,0.0,1.0


In [8]:
 # Import data

quarter= os.environ.get("DIVVY_QUARTER")
year= os.environ.get("DIVVY_YEAR")

raw_divvy_df = get_divvy_data(year,quarter)
raw_weather_df = get_weather_data()

print("Raw data imported")

# Clean data & merge data

clean_divvy_df = cleaning_divvy_gen_agg(raw_divvy_df)
clean_weather_df = weather_cleaning(raw_weather_df)

merged_df = merge_divvy_weather(clean_divvy_df, clean_weather_df)

Raw data imported


/Users/joachimclodic/code/G-Dolle/DIVVY_BIKE/ml_logic/cleaning.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_reduced.rename(columns={"name":"station_name"}, inplace=True)


In [9]:
merged_df.head()

,geohash,hourly_data,nb_departures,nb_arrivals,ratio,dt_iso,temp,pressure,humidity,wind_speed,wind_deg,clouds_all
0,dp3s,2022-01-08 13:00:00,1.0,0.0,inf,2022-01-08 13:00:00,-8.49,1022,68,5.66,160,20
1,dp3s,2022-01-12 15:00:00,1.0,0.0,inf,2022-01-12 15:00:00,1.79,1018,74,1.79,221,100
2,dp3s,2022-01-21 16:00:00,1.0,1.0,1.0,2022-01-21 16:00:00,-6.44,1037,61,3.09,220,20
3,dp3s,2022-01-28 17:00:00,1.0,1.0,1.0,2022-01-28 17:00:00,-6.43,1028,84,3.60,340,100
4,dp3s,2022-01-30 18:00:00,1.0,1.0,1.0,2022-01-30 18:00:00,-1.79,1017,73,3.58,352,83


In [10]:
merged_df.to_csv('merged_df.csv')

In [11]:
merged_df['geohash'].value_counts()

dp3w    2160
dp3t    2014
dp3x    1537
dp3v     110
dp3s      53
Name: geohash, dtype: int64

In [12]:
# X, y = features_target(merged_df, 'nb_departures')

# print("features and target dataframes created")

# # preprocess features
# preprocessor, X_processed_df = preprocess_features(X)


In [13]:
merged_df.dtypes

geohash                  object
hourly_data      datetime64[ns]
nb_departures           float64
nb_arrivals             float64
ratio                   float64
dt_iso           datetime64[ns]
temp                    float64
pressure                  int64
humidity                  int64
wind_speed              float64
wind_deg                  int64
clouds_all                int64
dtype: object

In [21]:
from sklearn.preprocessing import OneHotEncoder

ohe=OneHotEncoder()

# geohash_categories = {
#             0:'dp3w',
#             1:'dp3t',
#             2:'dp3x',
#             3:'dp3v',
#             4:'dp3s'}
    
OneHotEncoder(
#             categories=geohash_categories,
            sparse=False,
            handle_unknown="ignore")

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [22]:
ohe.fit(merged_df[['geohash']])

OneHotEncoder()

In [23]:
ohe.get_feature_names_out()

array(['geohash_dp3s', 'geohash_dp3t', 'geohash_dp3v', 'geohash_dp3w',
       'geohash_dp3x'], dtype=object)

In [24]:
ohe.transform(merged_df[['geohash']])

<5874x5 sparse matrix of type '<class 'numpy.float64'>'
	with 5874 stored elements in Compressed Sparse Row format>

In [33]:
merged_df[ohe.get_feature_names_out()] = ohe.transform(merged_df[['geohash']])

ValueError: Columns must be same length as key

In [ ]:
X_processed_df, y_processed_df, preprocessor=preprocess('nb_departures')
X_processed_df.head()

In [27]:
test_df=ohe.transform(merged_df[['geohash']])


In [32]:
test_df=pd.DataFrame(test_df)
test_df.head()[0]

0      (0, 0)\t1.0
1      (0, 0)\t1.0
2      (0, 0)\t1.0
3      (0, 0)\t1.0
4      (0, 0)\t1.0
Name: 0, dtype: object